In [1]:
import pandas as pd
import networkx as nx

features_friends = pd.read_csv("../data/FINAL_FEATURES_FRIENDS.tsv", sep="\t")
target_traintest = pd.read_csv("../data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep="\t")
sequences_matrix = pd.read_csv("../data/FINAL_SEQUENCES_MATRIX.tsv", sep="\t")
targets_traintest = pd.read_csv("../data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep="\t")

In [18]:
from torch_geometric.utils.convert import to_networkx, from_networkx
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx, from_networkx
from torch_geometric.nn import GCNConv
import torch


In [43]:
import sys
sys.path.append('../gcn')
from model import TwoLayerGCN, GCN, TwoLayerChebNet
from trainer import Trainer, RunConfig
import trainer as trainer
from utils import Dataset, load_data, set_labels

In [2]:
import numpy as np


targets_train=targets_traintest.loc[targets_traintest.TARGET!='test']

temp_df = features_friends.merge(target_traintest, on='CLIENT_ID', how = 'left')
temp_df = temp_df[:100]
features_columns = temp_df.drop(['CLIENT_ID', 'RETRO_DT','FRIEND_ID', 'TARGET'], axis = 1).columns

class stat:
    
    @staticmethod
    def get_mean(vector_features):
        embeddings_mean = list()
        for item in vector_features:
            embeddings_mean.append(np.mean(item))
        return embeddings_mean
        
        

In [3]:
%%time
attrs = {}
for i in range(len(temp_df)):
    tmp = {}
    for j in features_columns:
        tmp[j] = temp_df[j].loc[i]
        print(tmp)
    attrs[temp_df['FRIEND_ID'].loc[i]] = tmp
    tmp = {}
    

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [4]:
for item in pd.unique(temp_df['CLIENT_ID']):
    statistic_of_node = stat.get_mean(temp_df[temp_df['CLIENT_ID'] == item][features_columns].values.T)
    tmp = {}
    for i in range(len(features_columns)):
        tmp[features_columns[i]] = statistic_of_node[i]
    attrs[item] = tmp
    tmp = {}

In [5]:
G = nx.from_pandas_edgelist(temp_df, 'CLIENT_ID', 'FRIEND_ID')
nx.set_node_attributes(G, values = attrs)


In [8]:
g = from_networkx(G)

In [19]:
class SocialGNN(torch.nn.Module):
    def __init__(self,num_of_feat,f):
        super(SocialGNN, self).__init__()
        self.conv1 = GCNConv(num_of_feat, f)
        self.conv2 = GCNConv(f, 2)

    def forward(self, data):
        x = data.x.float()
        edge_index =  data.edge_index
        x = self.conv1(x=x, edge_index=edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [33]:
num_of_feat=g.num_node_features

In [34]:
num_of_feat

0

In [32]:
model = SocialGNN(

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   2,   3,   4,
           5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
          19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
          33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,
          47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
          61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
          75,  76,  77,  78,  79,  80,  81,  81,  81,  81,  81,  81,  81,  81,
          81,  81,  81,  81,  81,  81,  81,  81,  81

In [44]:
def evaluate_gcn_on_dataset(dataset: Dataset, iter = 1):
    set_seed()
    features, labels, train_labels, val_labels, test_labels, adjacency_matrix, \
    laplacian_matrix, num_classes = load_data(dataset)
    accuracies = []

    for i in range(iter):
        model = TwoLayerGCN(
            input_size=features.size(1),
            hidden_size=16,
            output_size=num_classes,
            dropout=0.5
        )
        trainer = Trainer(model)
        trainer.train(features, train_labels, val_labels, adjacency_matrix, device, run_config, log=False)

        _, accuracy = trainer.evaluate(features, test_labels, adjacency_matrix, device)
        accuracies.append(accuracy)
    print(f"\nPerformance on {dataset.name}:\n- test accuracy = {np.mean(accuracies):.3f} +- {np.std(accuracies):.3f}\n")

def evaluate_chebnet_on_dataset(dataset: Dataset, k = 2, iter = 1):
    set_seed()
    features, labels, train_labels, val_labels, test_labels, adjacency_matrix, \
    laplacian_matrix, num_classes = load_data(dataset)
    accuracies = []

    for i in range(iter):
        model = TwoLayerChebNet(
            input_size=features.size(1),
            hidden_size=16,
            output_size=num_classes,
            dropout=0.5,
            k=k
        )

        trainer = Trainer(model)
        trainer.train(features, train_labels, val_labels, laplacian_matrix, device, run_config, log=False)

        _ , accuracy = trainer.evaluate(features, test_labels, laplacian_matrix, device)
        accuracies.append(accuracy)
    print(f"\nPerformance on {dataset.name}:\n- test accuracy = {np.mean(accuracies):.3f} +- {np.std(accuracies):.3f}\n")

In [47]:
# important for reproducibility!
def set_seed(seed=1):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

In [52]:
g.get_tensor

<bound method FeatureStore.get_tensor of Data(edge_index=[2, 200], i1047=[102], i1048=[102], i1056=[102], i1058=[102], i1059=[102], i1060=[102], i1065=[102], i1066=[102], i1067=[102], i1068=[102], i1070=[102], i1073=[102], i1074=[102], i1075=[102], i1077=[102], i1079=[102], i1092=[102], i1095=[102], i1097=[102], i1130=[102], i1139=[102], i1147=[102], i1169=[102], i1193=[102], i1205=[102], i1214=[102], i1225=[102], i1228=[102], i1246=[102], i1248=[102], i1256=[102], i1262=[102], i1270=[102], i1280=[102], i1287=[102], i1297=[102], i1299=[102], i1328=[102], i1332=[102], i1339=[102], i1342=[102], i1343=[102], i1344=[102], i1345=[102], i1346=[102], i1347=[102], i1352=[102], i1353=[102], i1355=[102], i1357=[102], i1360=[102], i1361=[102], i1363=[102], i1364=[102], i1375=[102], i1386=[102], i1387=[102], i1392=[102], i1393=[102], i1411=[102], i1427=[102], i1431=[102], i1438=[102], i1442=[102], i1447=[102], i1449=[102], i1457=[102], i1460=[102], i1462=[102], i1463=[102], i1465=[102], i1466=[102